In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import pickle
from SistemaHierarquico import Sistema_Hierarquico_Base


In [4]:
with open("Models\FirstStage.p",'rb') as f:
    first_stage = pickle.load(f)
with open("Models\SecondStage.p","rb") as f:
    second_stage = pickle.load(f)

In [5]:



class SistemaHierarquico(Sistema_Hierarquico_Base):
        def __init__(self,anomaly_detector, classifier, n_classes, threshold_b, threshold_m, threshold_u):
            super().__init__(anomaly_detector, classifier, n_classes, threshold_b, threshold_m, threshold_u)

        def detect_anomaly_proba(self, X):
            # vetor com classe de maior probabilidade de cada instancia
            # pegam so a probabilidade dos elementos serem 1
            y_score = self.anomaly_detector.score_samples(X)

            # normaliza para ficar entre 0 e 1 como uma probabilidade
            y_score = (y_score - y_score.min())/(y_score.max() -y_score.min())

            # como valoores maiores sao de benignos, precisamos inverter
            y_score = 1-y_score
            return y_score

        def classify_proba(self,X):
            # matriz, cada linha representa uma instancia que pde ser maligna,
            # cada coluna representa uma classe
            # cada elemento da matriz representa uma probabilidade
            return self.classifier.predict_proba(X)

In [22]:
test_set = pd.read_csv("data\\test_full_model.csv")
#test_set = pd.read_csv("data\\test_dataset.csv")
test_set.Label.value_counts()

Label
BENIGN         100000
Brute Force      1100
(D)DOS           1076
Port Scan        1066
Botnet           1040
Web-Attack       1038
Name: count, dtype: int64

In [29]:
y_test = test_set.Label.to_numpy()
X_test = test_set.drop(columns= ["Label"]).to_numpy()

In [47]:
sh = SistemaHierarquico(first_stage,second_stage, 5, .25, .85, .6) 

In [48]:
y_pred = sh.predict(X_test)

d:\Disciplinas\intrusao\Projeto\Sistema_Hierarquico\Deteccao_de_Intrusao_Hierarquica_Multiestagio\env\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but QuantileTransformer was fitted with feature names
  warnings.warn(
d:\Disciplinas\intrusao\Projeto\Sistema_Hierarquico\Deteccao_de_Intrusao_Hierarquica_Multiestagio\env\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but QuantileTransformer was fitted with feature names
  warnings.warn(


In [49]:
second_stage.classes_

array(['(D)DOS', 'Botnet', 'Brute Force', 'Port Scan', 'Web-Attack'],
      dtype=object)

In [50]:
# basta usar o sescond.classes_ para anomalias, 
labels = np.array(['Unkown','(D)DOS', 'Botnet', 'Brute Force', 'Port Scan', 'Web-Attack', 'BENIGN'],dtype=str)
y_pred = np.array([labels[pred] for pred in y_pred])

In [51]:
print(classification_report(y_test,y_pred))

d:\Disciplinas\intrusao\Projeto\Sistema_Hierarquico\Deteccao_de_Intrusao_Hierarquica_Multiestagio\env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Disciplinas\intrusao\Projeto\Sistema_Hierarquico\Deteccao_de_Intrusao_Hierarquica_Multiestagio\env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

      (D)DOS       0.60      0.99      0.75      1076
      BENIGN       1.00      0.95      0.97    100000
      Botnet       0.45      1.00      0.62      1040
 Brute Force       0.75      0.97      0.85      1100
   Port Scan       1.00      1.00      1.00      1066
      Unkown       0.00      0.00      0.00         0
  Web-Attack       0.79      0.95      0.86      1038

    accuracy                           0.95    105320
   macro avg       0.66      0.84      0.72    105320
weighted avg       0.98      0.95      0.97    105320



d:\Disciplinas\intrusao\Projeto\Sistema_Hierarquico\Deteccao_de_Intrusao_Hierarquica_Multiestagio\env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
